# Module 1 Homework

In [ ]:
import math
import pickle

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.metrics import mean_squared_error 

In [ ]:
df = pd.read_parquet('data/yellow_tripdata_2023-01.parquet')

In [ ]:
df_len = len(df)
df_len

In [ ]:
print(f'January data has {len(df.columns)} columns.')

In [ ]:
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)

df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)

# Convert to minutes
df.duration = df.duration.apply(lambda x: x.total_seconds()/60.0)
df.duration.head()

In [ ]:
# Get std of duration
df.duration.describe()['std']

In [ ]:
# Remove outliers
df = df[(df.duration >= 1) & (df.duration <= 60)]

In [ ]:
df_len_after = len(df)
df_len_after

In [ ]:
# Percent of records retained after removing outliers
df_len_after/df_len*100

In [ ]:
df.head()

In [ ]:
categorical = ['PULocationID', 'DOLocationID']

df.loc[:, categorical] = df.loc[:, categorical].astype('Int64')
df.loc[:, categorical] = df.loc[:, categorical].astype(str)

df.dtypes

In [ ]:
dv = DictVectorizer()

train_dicts= df[categorical].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

In [ ]:
print(f'X_train has {X_train.shape[1]} colums.')

In [ ]:
# Target 
y_train = df.duration.values

In [ ]:
linear_model = LinearRegression()

linear_model.fit(X_train, y_train)

In [ ]:
y_pred = linear_model.predict(X_train)
print(f'Train RMSE: {math.sqrt(mean_squared_error(y_train, y_pred))}')

In [ ]:
# Function that does everythin above: Can be used for validation dataset

def process_dataset(path):
    df = pd.read_parquet(path)
    
    df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)
    df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
    
    df['duration'] = (df.tpep_dropoff_datetime - df.tpep_pickup_datetime)
    
    # Convert datetime to minutes
    df.duration = df.duration.apply(lambda x: x.total_seconds()/60.0)

    categorical = ['PULocationID', 'DOLocationID']

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    df.loc[:, categorical] = df.loc[:, categorical].astype('Int64')
    df.loc[:, categorical] = df.loc[:, categorical].astype(str)

    return df

In [ ]:
df_val = process_dataset('data/yellow_tripdata_2023-02.parquet')

In [ ]:
len(df_val)

In [ ]:
val_dicts = df_val[categorical].to_dict(orient='records')

X_val = dv.transform(val_dicts)
y_val = df_val.duration.values

In [ ]:
y_pred = linear_model.predict(X_val)
print(f'Val RMSE: {math.sqrt(mean_squared_error(y_val, y_pred))}')